In [ ]:
import pandas as pd
import numpy as np

# Load BERPublicsearch

In [ ]:
ber = pd.read_csv("../data/BERPublicsearch.zip")

In [ ]:
ber

# Bin into regulatory periods

See DEAP Manual 4.2.2 Appendix S: https://www.seai.ie/home-energy/building-energy-rating-ber/support-for-ber-assessors/domestic-ber-resources/

See Ciara Ahern PhD at: https://www.researchgate.net/profile/Ciara_Ahern

In [ ]:
age_bands = pd.read_csv("../data/age-bands.csv")

In [ ]:
age_bands

In [ ]:
ber["age_band_bin"] = pd.cut(
    ber["Year_of_Construction"],
    bins=[-np.inf, 1900, 1929, 1949, 1966, 1977, 1982, 1993, 1999, 2004, 2009, np.inf],
)

In [ ]:
age_band_bins = [age_band_bin for age_band_bin in ber["age_band_bin"].unique().sort_values()]

In [ ]:
age_bands = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]

In [ ]:
age_band_map = {age_band_bin: age_band  for age_band, age_band_bin in zip(age_bands, age_band_bins)}

In [ ]:
ber["age_band"] = ber["age_band_bin"].map(age_band_map)

In [ ]:
ber["age_band"]

# Wall default U-values

In [ ]:
wall_defaults_raw = pd.read_csv("../data/wall-uvalues.csv")

In [ ]:
wall_defaults_raw

In [ ]:
wall_defaults_pivoted = wall_defaults_raw.copy().melt(
    id_vars="Wall type",
    var_name="age_band",
    value_vars=age_bands,
    value_name="UValueWall_default"
)
wall_defaults_pivoted["wall_type"] = wall_defaults_pivoted["Wall type"].str.lower()
wall_defaults = wall_defaults_pivoted

In [ ]:
wall_defaults

In [ ]:
ber["FirstWallType_Description"].unique()

In [ ]:
ber["wall_type"] = ber["FirstWallType_Description"].fillna("Unknown").replace({"Other": "Unknown"}).str.lower()

In [ ]:
ber["wall_type"].unique()

In [ ]:
ber

In [ ]:
ber_merged = ber[["age_band", "wall_type", "UValueWall"]].merge(wall_defaults[["age_band", "wall_type", "UValueWall_default"]])

In [ ]:
ber_merged["UValueWall_is_default"] = ber_merged["UValueWall"] == ber_merged["UValueWall_default"]

In [ ]:
ber_uvalue_wall_is_default = ber_merged.query("`UValueWall_is_default` == True")

In [ ]:
ber_uvalue_wall_is_default

In [ ]:
number_of_uvalue_wall_default_hh = len(ber_uvalue_wall_is_default)
number_of_hh = len(ber)
percentage_of_uvalue_wall_default_hh = round(number_of_uvalue_wall_default_hh / number_of_hh * 100, 2)

In [ ]:
display(f"{percentage_of_uvalue_wall_default_hh}% of Dublin BERPublicsearch hh use default values for walls")